In [19]:
from data import *
from transforms import *
import os
import torch
import time
import random
from datetime import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import cv2
from skimage import io
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from catalyst.data.sampler import BalanceClassSampler

#CV2
import cv2

#Importing Tabnet
from pytorch_tabnet.tab_network import TabNet

#error
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score

from glob import glob
import sklearn

import pytorch_lightning as pl
import warnings

warnings.filterwarnings("ignore") 
warnings.filterwarnings("ignore", category=DeprecationWarning) 


%reload_ext autoreload
%autoreload 2

In [21]:
fold = 0
seed  = 42
tta   = 20

arch = 'efficientnet-b5'
resolution = 456  # orignal res for B5
input_res  = 512
lr = 8e-6   # * batch_size
weight_decay = 2e-5
pos_weight   = 3.2
label_smoothing = 0.03
max_epochs = 7

num_workers = 2  # os.cpu_count()
gpus = 1 if torch.cuda.is_available() else None

In [22]:
if gpus and not hasattr(torch.cuda, "amp"):
    try:
        from apex import amp
    except:
        !git clone https://github.com/NVIDIA/apex  nv_apex
        !pip3 install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./nv_apex
        from apex import amp

In [23]:
df = pd.read_csv(DATA / 'folds_13062020.csv').set_index('image_id')
df.head()

,patient_id,target,source,sex,age_approx,anatom_site_general_challenge,stratify_group,fold
image_id,,,,,,,,
ISIC_2637011,IP_7279968,0,ISIC20,male,45.0,head/neck,31,0
ISIC_0015719,IP_3075186,0,ISIC20,female,45.0,upper extremity,7,2
ISIC_0052212,IP_2842074,0,ISIC20,female,50.0,lower extremity,5,4
ISIC_0068279,IP_6890425,0,ISIC20,female,45.0,head/neck,7,0
ISIC_0074268,IP_8723313,0,ISIC20,female,55.0,upper extremity,6,4


In [26]:
features = ['sex', 'age_approx', 'anatom_site_general_challenge'] 
cat = ['sex', 'anatom_site_general_challenge']

categorical_columns = []

for col in cat:    
    print(col, df[col].nunique())        
    l_enc = LabelEncoder()
    df[col] = l_enc.fit_transform(df[col].values)

#     #SAVING LABEL _ ENC
#     output = open(f'{col}_encoder.pkl', 'wb')
#     pickle.dump(l_enc, output)
#     output.close()

    categorical_columns.append(col)

sex 3
anatom_site_general_challenge 8


In [27]:
df_test = pd.read_csv(DATA / 'test.csv', index_col='image_name')
df_test.head()

,patient_id,sex,age_approx,anatom_site_general_challenge,width,height
image_name,,,,,,
ISIC_0052060,IP_3579794,male,70.0,NaN,6000,4000
ISIC_0052349,IP_7782715,male,40.0,lower extremity,6000,4000
ISIC_0058510,IP_7960270,female,55.0,torso,6000,4000
ISIC_0073313,IP_6375035,female,50.0,torso,6000,4000
ISIC_0073502,IP_0589375,female,45.0,lower extremity,1920,1080


In [16]:
from torch.utils.data import Dataset, DataLoader

class ImageDataset(Dataset):
    def __init__(self, df, path, features, labels=None, transforms=None):
        super().__init__()
        self.df = df
        self.path = path
        self.features = self.df[features].
        self.labels = labels
        self.transforms = transforms

    def __getitem__(self, idx: int):
        image_id = self.df.index.values[idx]
        image = cv2.imread(f'{self.path}/{image_id}.jpg', cv2.IMREAD_COLOR)

        if self.transforms:
            sample = self.transforms(image=image)
            image  = sample['image']

        label = self.df.target.values[idx] if self.labels is not None else 0.5
        return image, label

    def __len__(self) -> int:
        return self.df.shape[0]

    def get_labels(self):
        return list(self.labels)
    
    @staticmethod
    def one_hot(size, target):
        tensor = torch.zeros(size, dtype=torch.float32)
        tensor[target] = 1.
        return tensor

In [15]:
ds_train = ImageDataset(
    path=TRAIN,
    image_ids=df[df_folds['fold'] != fold].index.values,
    labels=df[df['fold'] != fold].target.values,
    transforms=get_train_transforms(),
)

ds_val = ImageDataset(
    path=TRAIN,
    image_ids=df[df['fold'] == fold].index.values,
    labels=df[df_folds['fold'] == fold].target.values,
    transforms=get_valid_transforms(),
)

ds_test = ImageDataset(
    path=TEST,
    image_ids=df_test.index.values,
    transforms=get_tta_transforms(),
)

len(ds_train), len(ds_val), len(ds_test)

NameError: name 'ImageDataset' is not defined